In [119]:
# lots of aux functions
def isNonTerminal(c) :
    return c.isupper()

def isTerminal(c) :
    return not c.isupper()

def swap(s, i, j):
    lst = list(s)
    lst[i], lst[j] = lst[j], lst[i]
    return ''.join(lst)

In [120]:
class Production:
    def __init__(self,lhs,rhs):
        self.lhs = lhs
        self.rhs = rhs
    
    def __str__(self):
        return self.lhs + ' --> ' + self.rhs

    def __repr__(self):
        return self.__str__()

In [154]:
from dataclasses import dataclass
from typing import Any

@dataclass(frozen=True)
class DottedProduction :
    
    lhs : Any
    rhs : Any
    def __init__(self, lhs, rhs) :
        self.lhs = lhs
        self.rhs = rhs
                
    def __str__(self) :
        return self.lhs + ' --> ' + self.rhs
    
    def __repr__(self):
        return self.__str__()

    def __eq__(self, other) :
        return (self.lhs == other.lhs) and (self.rhs == other.rhs)
    
    def moveDot(self) :
        if self.isFinal() : return None
        return DottedProduction(self.lhs,self.move())
    
    def isFinal(self) :
        return self.rhs[-1] == '.'
    
    def dottedSymbol(self) :
        if self.isFinal() : return None
        return self.rhs[self.rhs.find('.') + 1]
    
    def move(self) :
        dotPos = self.rhs.find('.')
        return swap(self.rhs,dotPos,dotPos + 1)

In [122]:
p = DottedProduction('S','.aSa')
print(p)
print(p.dottedSymbol())
print(p.isFinal())
print(p.moveDot())

S --> .aSa
a
False
S --> a.Sa


In [123]:
class CFG:      
    def __init__(self, start, productions):
        self.start = start
        self.productions = productions
        
    def addProduction(self, production):
        self.productions.append(production)
        
    def productionsFor(self, nonTerminal) :
        return [p for p in self.productions if p.lhs == nonTerminal]
    
    def startProductions(self) :
        return self.productionsFor(self.start)

In [124]:
g = CFG('S',[])
g.addProduction(Production('S','E'))
g.addProduction(Production('E','E+T'))
g.addProduction(Production('E','T'))
g.addProduction(Production('T','n'))
print(g.productionsFor('E'))
print(g.startProductions())

[E --> E+T, E --> T]
[S --> E]


In [136]:
class SLR0Parser :
    
    def __init__(self, cfg):
        self.grammar = cfg
        self.itemCount = 0
        self.items = {} # empty dictionary
    
    # closure(p) where p is a dotted production
    def closure(self, dottedProduction) :
        result = {dottedProduction}
        symbol = dottedProduction.dottedSymbol()
        
        # Done if the dotted symbol is terminal
        if symbol is None or isTerminal(symbol) : return result
        
        # For all productions with a dotted NT on the LHS
        #    retrieve all grammar productions for this NT
        for p in self.grammar.productionsFor(symbol) :
            # Add the initial dotted production
            dottedP = DottedProduction(p.lhs,'.' + p.rhs)
            
            # recursively compute the closure and build the union
            if dottedP.dottedSymbol() != symbol : # watch for left recursion
                result  = result.union(self.closure(dottedP))
            else :
                # unless it's left recursive, of course
                result.add(dottedP)
        return result
    
    def transition(self, item, symbol) :
        # Create one item by picking all the dotted rules in "item",
        #   find those with a dot before "symbol" and
        #   create the set with all successors by moving the dot and closing it
        result = set()
        for production in itemSet :
            if production.dottedSymbol() == symbol :
                result = result.union(self.closure(production.moveDot()))
        return result
    
    def storeItem(self, item) :
        

In [155]:
p = SLR0Parser(g)
item1 = p.closure(DottedProduction('S','.E'))
item2 = p.closure(DottedProduction('S','.E'))
print(item1 == item2)

FrozenInstanceError: cannot assign to field 'lhs'

In [141]:
p = SLR0Parser(g)
item = p.closure(DottedProduction('S','.E'))
print(item)
print(p.transition(item,'E'))

{T --> .n, E --> .E+T, S --> .E, E --> .T}
{S --> E., E --> E.+T}
